In [ ]:
from google.colab import files
files = files.upload()

Saving tokenizer.pickle to tokenizer.pickle
Saving clean_df_test.csv to clean_df_test.csv
Saving clean_df_train.csv to clean_df_train.csv


In [ ]:
import sys

import tensorflow.keras
import pandas as pd
import sklearn as sk
import scipy as sp
import tensorflow as tf
import platform
import numpy as np 


print(f"Python Platform: {platform.platform()}")
print(f"Tensor Flow Version: {tf.__version__}")
print(f"Keras Version: {tensorflow.keras.__version__}")
print()
print(f"Python {sys.version}")
print(f"Pandas {pd.__version__}")
print(f"Scikit-Learn {sk.__version__}")
print(f"SciPy {sp.__version__}")
gpu = len(tf.config.list_physical_devices('GPU'))>0
print("GPU is", "available" if gpu else "NOT AVAILABLE")

Python Platform: Linux-5.10.147+-x86_64-with-glibc2.31
Tensor Flow Version: 2.11.0
Keras Version: 2.11.0

Python 3.9.16 (main, Dec  7 2022, 01:11:51) 
[GCC 9.4.0]
Pandas 1.4.4
Scikit-Learn 1.2.2
SciPy 1.10.1
GPU is available


In [ ]:
#modelling
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder,LabelEncoder
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM, Bidirectional, Embedding
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ReduceLROnPlateau

#saving model
import tensorflow_hub as hub

import matplotlib.pyplot as plt

tf.get_logger().setLevel('ERROR')


In [ ]:
DEVELOPING = True
MAX_LENGTH = 80

In [ ]:
!pip install spacy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import spacy
nlp = spacy.load("en_core_web_sm")#english tokenizer, tagger, parser and NER
def split_sentences(text):
    aux_sentence = nlp(text)
    return [str(token.lemma_).lower() for token in aux_sentence if not (token.is_stop or \
                                                                token.is_punct or \
                                                                len(token)<3)]

In [ ]:
data_train = pd.read_csv("clean_df_train.csv")
data_test = pd.read_csv("clean_df_test.csv")

In [ ]:
#subsampling for developing faster
if DEVELOPING:
    data_train = data_train.groupby('label').apply(lambda x: x.sample(frac=0.03)).reset_index(drop=True)
    data_test = data_test.groupby('label').apply(lambda x: x.sample(frac=0.03)).reset_index(drop=True)

In [ ]:
data_train["words_count"] = data_train.apply(lambda row: len(row["content"].split()), axis = 1)
data_test["words_count"] = data_test.apply(lambda row: len(row["content"].split()), axis = 1)

In [ ]:
data_train[data_train["words_count"] > 80]

,label,title,content,words_count
35,0.0,Velocity Inc.,Velocity Inc. is an American kit aircraft man...,81
67,0.0,Aker Floating Production,Aker Floating Production (OSE: AKFP) is a Nor...,85
167,0.0,Focus DIY,Focus DIY was a privately owned chain of DIY ...,82
169,0.0,Wall to Wall (production company),Wall to Wall part of the Shed Media Group is ...,123
229,0.0,Tombstone Records,Tombstone Records is an American record label...,81
...,...,...,...,...
11473,13.0,The Great Pyramid Robbery,The Great Pyramid Robbery is a fantasy novel ...,93
11486,13.0,The Mousehole Cat,The Mousehole Cat is a children's book writte...,87
11514,13.0,The Front Runner,The Front Runner is a 1974 novel by Patricia ...,84
11545,13.0,The Mirror Crack'd from Side to Side,The Mirror Crack'd from Side to Side is a wor...,87


In [ ]:
data_test[data_test["words_count"] > 80]

,label,title,content,words_count
25,0,Air Go Airlines,Air Go Airlines is a cargo airline based in A...,83
52,0,STA Travel,STA Travel is a travel company owned by the S...,81
94,0,Haudenosaunee Development Institute,Haudenosaunee Development Institute is the fo...,124
108,0,Halifax (Ireland),Bank of Scotland (Ireland) Limited was a bank...,85
111,0,Mixcrate,Mixcrate is an online audio distribution plat...,81
...,...,...,...,...
1294,12,A Time Out of War,A Time Out of War is a 1954 short war film di...,90
1316,12,About a Girl (film),About a Girl is a nine-minute short film dire...,82
1334,13,Loud and Quiet,Loud And Quiet is a free monthly music magazi...,81
1386,13,Three Problems for Solar Pons,Three Problems for Solar Pons is a collection...,88


In [ ]:
#parallelizing (Not sure it works correctly. Maybe mac hardware is not good for this)
import dask.dataframe as dd
from dask.multiprocessing import get

ddata = dd.from_pandas(data_train, npartitions=30)

#data_train[:1000].apply(lambda row: split_sentences(row["content"]), axis = 1)
ddata.map_partitions(lambda df: df.apply((lambda row: split_sentences(row["content"])), axis=1)).compute(scheduler="threads")

0        [bluerun, ventures, early, stage, venture, fir...
1        [avia, traffic, company, airline, head, office...
2        [fisher, communications, medium, company, unit...
3        [crossroads, pakistani, clothing, company, spe...
4        [crowdrise, profit, ecommerce, company, websit...
                               ...                        
11568    [doctor, dolittle, secret, lake, doctor, dolit...
11569    [eidolon, 2006, speculative, fiction, antholog...
11570    [burning, valley, 1953, come, age, novel, amer...
11571    [passion, michel, foucault, biography, french,...
11572    [unjust, war, moral, argument, historical, ill...
Length: 11573, dtype: object

In [ ]:
data_train

,label,title,content,words_count
0,0.0,BlueRun Ventures,BlueRun Ventures is an early stage venture fi...,24
1,0.0,Avia Traffic Company,Avia Traffic Company is an airline with its h...,33
2,0.0,Fisher Communications,Fisher Communications was a media company in ...,68
3,0.0,CrossRoads (clothing),CrossRoads is a Pakistani clothing company sp...,45
4,0.0,Crowdrise,CrowdRise is a for profit e-ecommerce company...,62
...,...,...,...,...
11568,13.0,Doctor Dolittle and the Secret Lake,Doctor Dolittle and the Secret Lake is a Doct...,64
11569,13.0,Eidolon I,Eidolon I is a 2006 speculative fiction antho...,16
11570,13.0,Burning Valley,Burning Valley is a 1953 coming-of-age novel ...,36
11571,13.0,The Passion of Michel Foucault,The Passion of Michel Foucault is a biography...,63


In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 51.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 24.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 92.8 MB/s eta 0:00:00


In [ ]:
import torch
torch.tensor([1,2,3], device="cuda")
from tqdm.notebook import tqdm

from transformers import BertTokenizer
from torch.utils.data import TensorDataset

from transformers import BertForSequenceClassification

In [ ]:
possible_labels = data_train.label.unique()

label_dict = {}
for index, possible_label in enumerate(possible_labels):
    label_dict[possible_label] = index
label_dict

{0.0: 0,
 1.0: 1,
 2.0: 2,
 3.0: 3,
 4.0: 4,
 5.0: 5,
 6.0: 6,
 7.0: 7,
 8.0: 8,
 9.0: 9,
 10.0: 10,
 11.0: 11,
 12.0: 12,
 13.0: 13}

In [ ]:
data_train['label'] = data_train.label.replace(label_dict)
data_test['label'] = data_test.label.replace(label_dict)


In [ ]:
label_names = {
    0.0: "Company",
    1: "EducationalInstitution",
    2: "Artist",
    3: "Athlete",
    4: "OfficeHolder",
    5: "MeanOfTransportation",
    6: "Building",
    7: "NaturalPlace",
    8: "Village",
    9: "Animal",
    10: "Plant",
    11: "Album",
    12: "Film",
    13: "WrittenWork"
}

In [ ]:
data_train

,label,title,content,words_count
0,0.0,BlueRun Ventures,BlueRun Ventures is an early stage venture fi...,24
1,0.0,Avia Traffic Company,Avia Traffic Company is an airline with its h...,33
2,0.0,Fisher Communications,Fisher Communications was a media company in ...,68
3,0.0,CrossRoads (clothing),CrossRoads is a Pakistani clothing company sp...,45
4,0.0,Crowdrise,CrowdRise is a for profit e-ecommerce company...,62
...,...,...,...,...
11568,13.0,Doctor Dolittle and the Secret Lake,Doctor Dolittle and the Secret Lake is a Doct...,64
11569,13.0,Eidolon I,Eidolon I is a 2006 speculative fiction antho...,16
11570,13.0,Burning Valley,Burning Valley is a 1953 coming-of-age novel ...,36
11571,13.0,The Passion of Michel Foucault,The Passion of Michel Foucault is a biography...,63


In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', 
                                          do_lower_case=True)

In [ ]:
encoded_data_train = tokenizer.batch_encode_plus(
    data_train.content.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    padding=True, 
    return_tensors='pt'
)

encoded_data_test = tokenizer.batch_encode_plus( 
    data_test.content.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    padding=True, 
    return_tensors='pt'
)


input_ids_train = encoded_data_train['input_ids']
attention_masks_train = encoded_data_train['attention_mask']
labels_train = torch.tensor(data_train.label.values)

input_ids_test = encoded_data_test['input_ids']
attention_masks_test = encoded_data_test['attention_mask']
labels_test = torch.tensor(data_test.label.values)

In [ ]:
tensor_train = TensorDataset(input_ids_train, attention_masks_train, labels_train)
tensor_test = TensorDataset(input_ids_test, attention_masks_test, labels_test)

In [ ]:
len(tensor_train), len(tensor_test)

(11573, 1443)

In [ ]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased",
                                                      num_labels=len(label_dict),
                                                      output_attentions=False,
                                                      output_hidden_states=False)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [ ]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

batch_size = 3

dataloader_train = DataLoader(tensor_train, 
                              sampler=RandomSampler(tensor_train), 
                              batch_size=batch_size)

dataloader_test = DataLoader(tensor_test, 
                                   sampler=SequentialSampler(tensor_test), 
                                   batch_size=batch_size)

In [ ]:
from transformers import get_linear_schedule_with_warmup
from torch.optim import AdamW

optimizer = AdamW(model.parameters(),
                  lr=1e-5, 
                  eps=1e-8)

In [ ]:
epochs = 5

scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps=0,
                                            num_training_steps=len(dataloader_train)*epochs)

In [63]:
from sklearn.metrics import f1_score

def f1_score_func(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = np.argmax(labels, axis=1).flatten()
    return f1_score(labels_flat, preds_flat, average='weighted')
def accuracy_per_class(preds, labels):
    label_dict_inverse = {v: k for k, v in label_dict.items()}
    
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = np.argmax(labels, axis=1).flatten()

    for label in np.unique(labels_flat):
        y_preds = preds_flat[labels_flat==label]
        y_true = labels_flat[labels_flat==label]
        print(f'Class: {label_dict_inverse[label]}')
        print(f'Accuracy: {len(y_preds[y_preds==label])}/{len(y_true)}\n')

In [53]:
import random

seed_val = 17
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)

In [54]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

print(device)

cuda


In [55]:
import os
if not os.path.exists('data_volume'):
    os.makedirs('data_volume')

In [56]:
def one_hot_encoding(labels, num_classes):
    batch_size = labels.size(0)
    one_hot_labels = torch.zeros(batch_size, num_classes).to(device)
    one_hot_labels.scatter_(1, labels.unsqueeze(1).to(torch.int64), 1.0)
    return one_hot_labels
num_classes = 14

In [57]:
def evaluate(dataloader_val):

    model.eval()
    
    loss_val_total = 0
    predictions, true_vals = [], []
    
    for batch in dataloader_val:
        
        batch = tuple(b.to(device) for b in batch)
  
        
        one_hot_labels = one_hot_encoding(batch[2], num_classes)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         one_hot_labels,
                 }

        with torch.no_grad():        
            outputs = model(**inputs)
            
        loss = outputs[0]
        logits = outputs[1]
        loss_val_total += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)
    
    loss_val_avg = loss_val_total/len(dataloader_val) 
    
    predictions = np.vstack(predictions)
    true_vals = np.vstack(true_vals)
            
    return loss_val_avg, predictions, true_vals


In [58]:

for epoch in tqdm(range(1, epochs+1)):
    model.train()
    
    loss_train_total = 0

    progress_bar = tqdm(dataloader_train, desc='Epoch {:1d}'.format(epoch), leave=False, disable=False)
    for batch in progress_bar:

        model.zero_grad()
        batch = tuple(b.to(device) for b in batch)
        one_hot_labels = one_hot_encoding(batch[2], num_classes)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         one_hot_labels,
                 }        

        outputs = model(**inputs)
        loss = outputs[0]
        loss_train_total += loss.item()
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()
        scheduler.step()
        progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})
        
    torch.save(model.state_dict(), f'data_volume/finetuned_BERT_epoch_{epoch}.model')
        
    tqdm.write(f'\nEpoch {epoch}')
    
    loss_train_avg = loss_train_total/len(dataloader_train)            
    tqdm.write(f'Training loss: {loss_train_avg}')
    
    val_loss, predictions, true_vals = evaluate(dataloader_test)
    val_f1 = f1_score_func(predictions, true_vals)
    tqdm.write(f'Validation loss: {val_loss}')
    tqdm.write(f'F1 Score (Weighted): {val_f1}')

  0%|          | 0/5 [00:00<?, ?it/s]

Epoch 1:   0%|          | 0/3858 [00:00<?, ?it/s]


Epoch 1
Training loss: 0.001192403908454687
Validation loss: 0.012202339686149356
F1 Score (Weighted): 0.9861390953626342


Epoch 2:   0%|          | 0/3858 [00:00<?, ?it/s]


Epoch 2
Training loss: 0.0012977393032638225
Validation loss: 0.012202339686149356
F1 Score (Weighted): 0.9861390953626342


...


Epoch 3
Training loss: 0.0012257608625101926
Validation loss: 0.012202339686149356
F1 Score (Weighted): 0.9861390953626342


Epoch 4:   0%|          | 0/3858 [00:00<?, ?it/s]


Epoch 4
Training loss: 0.0012420055126585935
Validation loss: 0.012202339686149356
F1 Score (Weighted): 0.9861390953626342


Epoch 5:   0%|          | 0/3858 [00:00<?, ?it/s]


Epoch 5
Training loss: 0.001238439743140444
Validation loss: 0.012202339686149356
F1 Score (Weighted): 0.9861390953626342


In [59]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased",
                                                      num_labels=len(label_dict),
                                                      output_attentions=False,
                                                      output_hidden_states=False)

model.to(device)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [60]:
model.load_state_dict(torch.load('data_volume/finetuned_BERT_epoch_1.model', map_location=torch.device('cuda')))

<All keys matched successfully>

In [61]:
_, predictions, true_vals = evaluate(dataloader_test)


In [64]:
accuracy_per_class(predictions, true_vals)


Class: 0.0
Accuracy: 111/117

Class: 1.0
Accuracy: 114/116

Class: 2.0
Accuracy: 79/81

Class: 3.0
Accuracy: 80/80

Class: 4.0
Accuracy: 73/73

Class: 5.0
Accuracy: 120/121

Class: 6.0
Accuracy: 119/120

Class: 7.0
Accuracy: 84/86

Class: 8.0
Accuracy: 25/26

Class: 9.0
Accuracy: 134/134

Class: 10.0
Accuracy: 130/132

Class: 11.0
Accuracy: 129/130

Class: 12.0
Accuracy: 108/110

Class: 13.0
Accuracy: 117/117

